In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils import resample
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
import warnings

warnings.filterwarnings('ignore')

print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\kevin\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
cuda


In [2]:
learning_rate = 0.0005
weight_decay = 0.0001       # L2 regularization appled to the optimizer
epochs = 40

hidden_size = 16
num_layers = 8

In [3]:
user1 = pd.read_csv('user1_preprocessed_4.csv')
user2 = pd.read_csv('user2_preprocessed_4.csv')

user1.drop(columns='Time', inplace=True)
user2.drop(columns='Time', inplace=True)

user2['seq_num'] = user2['seq_num'] + user1[-1:]['seq_num'].values + 1

user = pd.concat([user1, user2], axis=0)

print(user)
print(user.shape)
print(user1.shape)
print(user2.shape)

            Event_Type    X    Y  PAM_Val  seq_num  delta_time
0                 Move  518  381        4        0           0
1                 Move  511  388        4        0     7980000
2                 Move  509  393        4        0     7977000
3                 Move  505  397        4        0     7978000
4                 Move  501  399        4        0     7979000
...                ...  ...  ...      ...      ...         ...
4272853           Move  680  831        3       97    76169000
4272854           Move  677  831        3       97    25399000
4272855           Move  677  829        3       97     3894000
4272856   Left_Pressed  677  829        3       97    75190000
4272857  Left_Released  677  829        3       97    97661000

[5117079 rows x 6 columns]
(5117079, 6)
(844221, 6)
(4272858, 6)


In [4]:
app_one_hot_encoded = pd.get_dummies(user['Event_Type'])

user.drop(columns='Event_Type', inplace=True)

user = pd.concat([user, app_one_hot_encoded], axis=1)

print(user.head(5))

     X    Y  PAM_Val  seq_num  delta_time  Left_Pressed  Left_Released  Move  \
0  518  381        4        0           0         False          False  True   
1  511  388        4        0     7980000         False          False  True   
2  509  393        4        0     7977000         False          False  True   
3  505  397        4        0     7978000         False          False  True   
4  501  399        4        0     7979000         False          False  True   

   Right_Pressed  Right_Released  Scroll  
0          False           False   False  
1          False           False   False  
2          False           False   False  
3          False           False   False  
4          False           False   False  


In [5]:
print(user.shape)

(5117079, 11)


In [6]:
user['PAM_Val'] = user['PAM_Val'] - 1.0

print(user.head(5))

     X    Y  PAM_Val  seq_num  delta_time  Left_Pressed  Left_Released  Move  \
0  518  381      3.0        0           0         False          False  True   
1  511  388      3.0        0     7980000         False          False  True   
2  509  393      3.0        0     7977000         False          False  True   
3  505  397      3.0        0     7978000         False          False  True   
4  501  399      3.0        0     7979000         False          False  True   

   Right_Pressed  Right_Released  Scroll  
0          False           False   False  
1          False           False   False  
2          False           False   False  
3          False           False   False  
4          False           False   False  


In [7]:
user_X = user.drop(columns=['PAM_Val'])
user_y = user.filter(['PAM_Val', 'seq_num'])

print(user_X.head(5))
print(user_y.head(5))

     X    Y  seq_num  delta_time  Left_Pressed  Left_Released  Move  \
0  518  381        0           0         False          False  True   
1  511  388        0     7980000         False          False  True   
2  509  393        0     7977000         False          False  True   
3  505  397        0     7978000         False          False  True   
4  501  399        0     7979000         False          False  True   

   Right_Pressed  Right_Released  Scroll  
0          False           False   False  
1          False           False   False  
2          False           False   False  
3          False           False   False  
4          False           False   False  
   PAM_Val  seq_num
0      3.0        0
1      3.0        0
2      3.0        0
3      3.0        0
4      3.0        0


In [8]:
user_X = user_X.groupby(by='seq_num')
user_y = user_y.groupby(by='seq_num')

In [9]:
X = []
y = []

for one, two in zip(user_X, user_y):
    X.append(pd.concat([one[1].drop(columns=['seq_num']), two[1]['PAM_Val'].shift(1)], axis=1).fillna(0).iloc[-100:].values)
    y.append(two[1].drop(columns=['seq_num']).iloc[-5000:].values)

print(len(X))
print(len(X[0]))

98
100


In [10]:
# the pam value for each timestep in a sequence is the same, so condense them into a single value per sequence

print(y[0])

for index in range(len(y)):
    y[index] = y[index][-1]

print(y[0])

[[3.]
 [3.]
 [3.]
 ...
 [3.]
 [3.]
 [3.]]
[3.]


In [11]:
class RNN_Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    super(RNN_Model, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.output_size = output_size

    self.rnn = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True, dropout=0.3)
    self.fc = nn.Linear(self.hidden_size, self.output_size)
    #self.sm = nn.Softmax(dim=0)    # not needed if using Cross Entropy loss
  
  def forward(self, x):
    batch_size = x.size(0)

    h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

    out, hidden = self.rnn(x, (h0, c0))
    #out = self.fc(out.contiguous().view(-1, self.hidden_size))
    out = self.fc(out[:, -1, :])
    #out = self.sm(out)

    return out, hidden

In [12]:
unique, counts = np.unique(y, return_counts=True)
print(np.asarray((unique, counts)))

weighted_classes = []
minimum_count = min(counts)
for value, count in zip(unique, counts):
    weighted_classes.append(minimum_count / count)

weighted_classes = torch.Tensor(weighted_classes).to(device)
print(weighted_classes)

[[ 0.  1.  2.  3.]
 [29. 47. 14.  8.]]
tensor([0.2759, 0.1702, 0.5714, 1.0000], device='cuda:0')


In [13]:
loss_fn = nn.CrossEntropyLoss(weight=weighted_classes)

In [14]:
kfold = StratifiedKFold(n_splits=3, shuffle=True)

In [15]:
comb_accuracy = []
comb_precision = []
comb_recall = []
comb_f1 = []
comb_accuracy_train = []
comb_precision_train = []
comb_recall_train = []
comb_f1_train = []
for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    print(f"======================== Fold {i} ========================")

    X_train = np.array(X)[train_index].tolist()
    y_train = np.array(y)[train_index].tolist()

    X_test = np.array(X)[test_index].tolist()
    y_test = np.array(y)[test_index].tolist()

    class_weights = []

    # ===== train ===== #
    model = RNN_Model(len(X_train[0][0]), hidden_size, 4, num_layers)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    for epoch in range(epochs):
      model.train()
      train_loss = 0.0
      for sample, label in zip(X_train, y_train):
        sample = torch.Tensor([sample]).to(device)
        label = torch.Tensor(label).type(torch.cuda.LongTensor).to(device)

        outputs, hidden = model(sample)

        loss = loss_fn(outputs, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss

      train_loss /= len(X_train)

      model.eval()
      val_loss = 0.0
      with torch.no_grad():
        for sample, label in zip(X_test, y_test):
          sample = torch.Tensor([sample]).to(device)
          label = torch.Tensor(label).type(torch.cuda.LongTensor).to(device)

          outputs, hidden = model(sample)
          loss = loss_fn(outputs, label)
          val_loss += loss
      
      val_loss /= len(X_test)
      scheduler.step(val_loss)

      print(f'epoch {epoch+1}/{epochs}: train loss = {train_loss:.4f}, val loss = {val_loss:.4f}')



    model.eval()

    # ===== metrics on training splits ===== #
    conf_matrix_train = []
    accuracy_train = []
    precision_train = []
    recall_train = []
    f1_train = []
    with torch.no_grad():
      for sample, label in zip(X_train, y_train):
        sample = torch.Tensor([sample]).to(device)
        label = torch.Tensor(label).type(torch.cuda.LongTensor).to(device)
        label = label.to('cpu')

        outputs, hidden = model(sample)
        _, y_pred_train = torch.max(outputs, 1)  # returns value, index

        y_pred_train_cpu = y_pred_train.to('cpu')

        conf_matrix_train.append(confusion_matrix(label, y_pred_train_cpu, labels=[0,1,2,3]))
        accuracy_train.append(accuracy_score(label, y_pred_train_cpu))
        precision_train.append(precision_score(label, y_pred_train_cpu, labels=[0,1,2,3], average='macro'))
        recall_train.append(recall_score(label, y_pred_train_cpu, labels=[0,1,2,3], average='macro'))
        f1_train.append(f1_score(label, y_pred_train_cpu, labels=[0,1,2,3], average='macro'))
    
    conf_matrix_train = np.array(conf_matrix_train).sum(axis=0).tolist()
    accuracy_train = np.array(accuracy_train).mean(axis=0).tolist()
    precision_train = np.array(precision_train).mean(axis=0).tolist()
    recall_train = np.array(recall_train).mean(axis=0).tolist()
    f1_train = np.array(f1_train).mean(axis=0).tolist()

    # ===== metrics on test split(s) ===== #
    conf_matrix = []
    accuracy = []
    precision = []
    recall = []
    f1 = []
    with torch.no_grad():
      for sample, label in zip(X_test, y_test):
        sample = torch.Tensor([sample]).to(device)
        label = torch.Tensor(label).type(torch.cuda.LongTensor).to(device)
        label = label.to('cpu')

        outputs, hidden = model(sample)
        _, y_pred = torch.max(outputs, 1)  # returns value, index

        y_pred_cpu = y_pred.to('cpu')

        conf_matrix.append(confusion_matrix(label, y_pred_cpu, labels=[0,1,2,3]))
        accuracy.append(accuracy_score(label, y_pred_cpu))
        precision.append(precision_score(label, y_pred_cpu, labels=[0,1,2,3], average='macro'))
        recall.append(recall_score(label, y_pred_cpu, labels=[0,1,2,3], average='macro'))
        f1.append(f1_score(label, y_pred_cpu, labels=[0,1,2,3], average='macro'))
    
    conf_matrix = np.array(conf_matrix).sum(axis=0).tolist()
    accuracy = np.array(accuracy).mean(axis=0).tolist()
    precision = np.array(precision).mean(axis=0).tolist()
    recall = np.array(recall).mean(axis=0).tolist()
    f1 = np.array(f1).mean(axis=0).tolist()



    comb_accuracy_train.append(accuracy_train)
    comb_precision_train.append(precision_train)
    comb_recall_train.append(recall_train)
    comb_f1_train.append(f1_train)

    comb_accuracy.append(accuracy)
    comb_precision.append(precision)
    comb_recall.append(recall)
    comb_f1.append(f1)

    print('Confusion Matrix:')
    print(conf_matrix_train)
    print('Accuracy:')
    print(accuracy_train)
    print('Precision:')
    print(precision_train)
    print('Recall:')
    print(recall_train)
    print('F1:')
    print(f1_train)
    print()

======================== Fold 0 ========================
epoch 1/40: train loss = 1.2931, val loss = 1.2930
epoch 2/40: train loss = 1.2681, val loss = 1.2745
epoch 3/40: train loss = 1.2448, val loss = 1.2537
epoch 4/40: train loss = 1.2190, val loss = 1.2359
epoch 5/40: train loss = 1.2016, val loss = 1.2329
epoch 6/40: train loss = 1.1969, val loss = 1.2316
epoch 7/40: train loss = 1.1958, val loss = 1.2304
epoch 8/40: train loss = 1.1885, val loss = 1.2308
epoch 9/40: train loss = 1.1933, val loss = 1.2301
epoch 10/40: train loss = 1.1880, val loss = 1.2302
epoch 11/40: train loss = 1.1862, val loss = 1.2307
epoch 12/40: train loss = 1.1925, val loss = 1.2298
epoch 13/40: train loss = 1.1907, val loss = 1.2293
epoch 14/40: train loss = 1.1867, val loss = 1.2292
epoch 15/40: train loss = 1.1855, val loss = 1.2294
epoch 16/40: train loss = 1.1862, val loss = 1.2293
epoch 17/40: train loss = 1.1892, val loss = 1.2289
epoch 18/40: train loss = 1.1851, val loss = 1.2289
epoch 19/40: tra

In [16]:
comb_accuracy_train = np.array(comb_accuracy_train)
comb_precision_train = np.array(comb_precision_train)
comb_recall_train = np.array(comb_recall_train)
comb_f1_train = np.array(comb_f1_train)

comb_accuracy = np.array(comb_accuracy)
comb_precision = np.array(comb_precision)
comb_recall = np.array(comb_recall)
comb_f1 = np.array(comb_f1)

print('Average Accuracy:')
print(comb_accuracy_train.mean(axis=0))
print(comb_accuracy.mean(axis=0))
print('Average Precision:')
print(comb_precision_train.mean(axis=0))
print(comb_precision.mean(axis=0))
print('Average Recall:')
print(comb_recall_train.mean(axis=0))
print(comb_recall.mean(axis=0))
print('Average F1:')
print(comb_f1_train.mean(axis=0))
print(comb_f1.mean(axis=0))

Average Accuracy:
0.47964257964257967
0.47979797979797983
Average Precision:
0.11991064491064492
0.11994949494949496
Average Recall:
0.11991064491064492
0.11994949494949496
Average F1:
0.11991064491064492
0.11994949494949496
